In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import cv2

In [2]:
# Load the MNIST dataset
mnist = fetch_openml('mnist_784')

# Preprocess the data
X = mnist.data.astype('float32') / 255.0
y = mnist.target.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model architecture
model = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', solver='adam', max_iter=5)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model on the test set
accuracy = model.score(X_test, y_test)
print("Test accuracy:", accuracy)

/Users/lovecoder/Library/Python/3.9/lib/python/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Test accuracy: 0.9673571428571428


/Users/lovecoder/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# Load the input image and convert it to grayscale
image = cv2.imread('testing.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold the image to extract the handwritten text
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours in the image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate over the contours and extract each individual character
output = []
for contour in contours:
    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)

    # Extract the character from the image
    roi = thresh[y:y+h, x:x+w]

    # Resize the character to 28x28 (same size as MNIST images)
    resized = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)

    # Reshape the image to a 2D tensor to match the input shape of the model
    reshaped = np.reshape(resized, (1, 784))

    # Make a prediction with the model
    prediction = model.predict(reshaped)

    # Get the predicted digit and append it to the output list
    digit = np.argmax(prediction)
    output.append(str(digit))

# Join the output list into a string
recognized_text = ''.join(output)

# Print the recognized text
print("Recognized text:", recognized_text)
